# Homefinder Simulation

Import necessary libraries

In [17]:
import random
import datetime 
import pandas as pd
from faker import Faker

Set random generation seed

In [18]:
random.seed(42)

## Creating class-objects for Houses and Applicants

In [19]:
class House:
    shortlist = pd.DataFrame(columns=['id','priority','bp_date','ap_date'])
    def __init__(self,id,rooms,status,bids,location='',rent=0,slist=shortlist,tenant=0):
        self.id = id
        self.rooms = rooms
        self.status = status 
        self.bids = bids
        self.slist = slist
        self.tenant = tenant
        self.rent = rent # currently not used 
        self.location = location # currently not used
    
    def __str__(self):
        return f"House ID:{self.id} has {self.rooms} rooms and {len(self.bids)} bid(s). It is currently {self.status}"


    # function to shortlist applicants and choose top applicant and send offer
    def shortlister(self):
        shortlist = pd.DataFrame(columns=['id','priority','bp_date','ap_date'])
        ## shortlisted in order of priority band, then effective band date, then application date
        for x in self.bids: # checks bids and applicant ids to generate shortlist
            if applicants[x][0].status == 'Live':
                shortlist.loc[x,'id'] = applicants[x][0].id
                shortlist.loc[x,'priority'] = applicants[x][0].priority
                shortlist.loc[x,'bp_date'] = applicants[x][0].bp_date
                shortlist.loc[x, 'ap_date'] = applicants[x][0].ap_date
            else:
                continue
        
        shortlist.sort_values(['priority','bp_date','ap_date'],axis=0,inplace=True,ignore_index=True)
        self.slist = shortlist
        if len(self.slist) < 1:
            self.status = 'No bids'
            return self.status
        else:
            top = self.slist.loc[0,'id']
            applicants[top][1].append(self.id)
            #print(top)
            return shortlist
    
    # function to sign contract with top applicant if top applicant accepts offer
    def contract(self):
        top = self.slist.loc[0,'id']
        if applicants[top][0].home == self.id:
            # change house and applicant status from 'Live' to 'Occupied'/'Housed' to remove from bidding process
            self.status = 'Occupied'
            self.tenant = top
            applicants[top][0].status = 'Housed'
        else:
            pass



            
        
        

In [20]:
class Applicant:
    bid = []
    def __init__(self,id,priority,bp_date,ap_date,hsize,status,offers,income=0,location='',bids=bid,home=0):
        self.id = id
        self.priority = priority
        self.bp_date = bp_date
        self.ap_date = ap_date
        self.hsize = hsize
        self.status = status
        self.bids = bids
        self.offers = offers
        self.home = home
        self.income = income # currently not used
        self.location = location # currently not used

    def __str__(self):
        return f"Applicant {self.id} is in Priority Band {self.priority}. They have a need of {self.hsize} rooms and are currently {self.status}"
    
    # function for bidding logic - applicants choose homes which are the same size as their household need and pick 3 randomly (3 bids), this 
    # should be modified later to choose homes based on logic instead of random
    def bidder(self,hlist):
        shortlist = []
        for x in hlist:
            if hlist[x].rooms == self.hsize and hlist[x].status == 'Live':
                shortlist.append((x,hlist[x]))
        try:
            bids = random.sample(shortlist,k=3)
        except ValueError:
            bids = random.sample(shortlist,k=len(shortlist)) # if the number of suitable properties are less than 3
        for x in bids:
            houses[x[0]].bids.append(self.id) # adds applicant id to list of bids at house they are applying in
        self.bids = bids 
        return bids
    
    # checks offers from homes they bid on, and picks one randomly, this should be modified later to add complexity and decide homes based on
    # some logic.
    def shortlist(self): 
        if len(self.offers) > 1:
            self.home = random.choice(self.offers)
            return self.home
        elif len(self.offers) == 1:
            self.home = self.offers[0]
            return self.home
        else:
            pass
        
    


## Generate random applicants and houses

Create generator functions to create homes and applicants, currently randomly generated, can use actual datasets to create synthetic homes and applicants based on real data

In [21]:
fake = Faker()
Faker.seed(42)
def date_generator(y,m,d):
    start_date = datetime.datetime(y,m,d)
    return fake.date_between(start_date=start_date, end_date='now')

def house_generator(ind):
    id = ind
    rooms  = random.randint(0,5)
    house = House(id,rooms,'Live',[])
    return house

def applicant_generator(ind):
    id = ind
    priority = random.randint(1,4)
    bp_date = date_generator(2015,1,1)
    ap_date = date_generator(2021,9,19)
    hsize = random.randint(0,5)
    offers = []
    applicant = Applicant(id,priority,bp_date,ap_date,hsize,offers=offers,status='Live')
    return applicant

To rerun, delete homes and applicants before regeneration. House and applicant objects stored in dictionary with ID as key

In [22]:
try:
    del houses
except NameError:
    pass
houses = {}
number_of_houses = input('Number of houses to generate: ')
for x in range(1,int(number_of_houses)): # change upper limit to change number of homes generated
    houses[x] = house_generator(x)

In [23]:
try:
    del applicants
except NameError:
    pass
applicants = {}
number_of_applicants = input('Number of applicants to generate: ')
for x in range(1,int(number_of_applicants)): # change upper limit to change number of applicant generated
    applicants[x] = [applicant_generator(x),[]]

A typical biddinng cycle consists of a round of bidding, then landlords shortlist tenants and make an offer to the top applicant, the applicant picks which home to move to if they receive multiple offers. The landlord then picks the next most suitable tenant, and the cycle continues till all houses are occupied

Creating dataframe to check homes list

In [24]:
homes = pd.DataFrame(columns=['id','rooms','status'])
for x in houses:
    homes.loc[x,'id'] = houses[x].id
    homes.loc[x,'rooms'] = houses[x].rooms
    homes.loc[x,'status'] = houses[x].status

In [25]:
for x in houses:
    homes.loc[x,'status'] = houses[x].status
homes

,id,rooms,status
1,1,5,Live
2,2,0,Live
3,3,0,Live
4,4,5,Live
5,5,2,Live
...,...,...,...
195,195,3,Live
196,196,0,Live
197,197,0,Live
198,198,2,Live


To check number of homes still listed

In [26]:
homes.groupby('status').count()

,id,rooms
status,,
Live,199,199


The bidding cycle simulation, runs until either all homes are occupied or 20 cycles have been run

In [27]:
cycle = 0
while 'Live' in homes['status'].unique():
    cycle += 1
    print(cycle)
    for x in applicants:
        if applicants[x][0].status == 'Live':
            applicants[x][0].bidder(houses)
        else:
            continue
    for x in houses:
        if houses[x].status != 'Occupied':
            houses[x].shortlister()
        else:
            continue
    for x in applicants:
        applicants[x][0].offers = applicants[x][1]
        if applicants[x][0].status == 'Live' and len(applicants[x][0].offers) > 1:
            applicants[x][0].shortlist()
            house = applicants[x][0].home
            houses[house].contract()
        else:
            continue
    for x in houses:
        homes.loc[x,'status'] = houses[x].status
    
    if cycle > 20:
        print('Limit reached, stopping simulation.')
        display(homes.groupby('status').count())
        break
else:
    display(homes.groupby('status').count())

                


1
2
3
4
5
6
7
8
9
10
11
12
13


,id,rooms
status,,
Occupied,199,199
